In [15]:
#Get search endpoint
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
import os

load_dotenv()

ai_search_endpoint = os.getenv("AI_SEARCH_ENDPOINT")
ai_search_apikey = os.getenv("AI_SEARCH_KEY")
ai_search_index_name = os.getenv("AI_SEARCH_INDEX_NAME")
ai_search_credential = AzureKeyCredential(ai_search_apikey)
search_client = SearchClient(endpoint=ai_search_endpoint, index_name=ai_search_index_name, credential=ai_search_credential)

In [17]:
#Get Generative model
from openai import AzureOpenAI

endpoint = os.getenv("FOUNDRY_ENDPOINT")
model_name = os.getenv("GENERATIVE_MODEL_NAME")
subscription_key = os.getenv("FOUNDRY_KEY")
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

In [18]:
#Get embedding model
embedding_model_name = os.getenv("EMBEDDING_MODEL_NAME")
embedding_client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key
)

In [16]:
from azure.search.documents.models import VectorizedQuery
query = "How to fix my router?"
k=5

#get query embedding
query_embed = embedding_client.embeddings.create(input=query, model=embedding_model_name).data[0].embedding

#create vector query
vector_query = VectorizedQuery(
    vector=query_embed,
    k_nearest_neighbors=k,
    fields='embeddingContent',
    kind="vector",
    exhaustive=True
)
#search for most relevant documents
results = search_client.search(
    search_text="*",      # ignored for pure vector search
    vector_queries=[vector_query],
    select=['title', 'content'],
    top=k
)

#Generate answer based on retrieved documents
contents = []
for r in results:
    contents.append(r["content"])

context = "\n\n---\n".join(contents)

messages = [
    {
        "role": "system",
        "content": (
            "You are an AI assistant. Use ONLY the provided context to answer. "
            "If the answer is not in the context, say you don't know.\n\n"
            f"CONTEXT:\n{context}"
        )
    },
    {
        "role": "user",
        "content": query
    }
]

response = client.chat.completions.create(
    model=model_name,   # or your Azure deployment name
    messages=messages,
    max_tokens=400,
    temperature=0.1
)
answer = response.choices[0].message.content
print(answer)

To fix your router, you can follow these steps:

1. Restart the Router:
   - Turn off the router.
   - Wait for about 10-30 seconds.
   - Turn it back on.

2. Check Connections:
   - Ensure the router is properly connected to power and your modem.
   - Verify that cables are securely plugged in.

3. Access Router Settings:
   - Connect a computer to the router via Ethernet or Wi-Fi.
   - Open a web browser and enter the router's IP address (commonly 192.168.0.1, 192.168.1.1, or 10.0.0.1).
   - Log in using the default credentials found on the router sticker or manual (e.g., username: admin, password: admin).

4. Update Firmware:
   - In the router settings, check for firmware updates and install the latest version.

5. Reset to Factory Settings:
   - If login credentials don't work or issues persist, press and hold the reset button on the router for 10-15 seconds to restore factory defaults.

6. Configure Basic Settings:
   - Set your Wi-Fi network name (SSID) and a strong password.
  

In [7]:
from azure.search.documents.models import VectorizedQuery
def search_documents(query, embedding_client, embedding_model_name, search_client, search_fields,selected_fields,k):
    #get query embedding
    query_embed = embedding_client.embeddings.create(input=query, model=embedding_model_name).data[0].embedding
    
    #create vector query
    vector_query = VectorizedQuery(
        vector=query_embed,
        k_nearest_neighbors=k,
        fields=search_fields,
        kind="vector",
        exhaustive=True
    )
    #search for most relevant documents
    results = search_client.search(
        search_text="*",      # ignored for pure vector search
        vector_queries=[vector_query],
        select=selected_fields,
        top=k
    )
    
    #Generate answer based on retrieved documents
    documents = {}
    for field in selected_fields:
        documents[field] = []
    for r in results:
        for field in selected_fields:
            documents[field].append(r[field])
    return documents

def generate_answer(documents, chat_client, chat_model_name):
    #create context blocks: assume title and content
    titles = documents["title"]
    contents = documents["content"]
    context_blocks = []
    for content, title in zip(contents,titles):
        block = f"###{title}\n{content}"
        context_blocks.append(block)

    #convert into one context
    context = "\n\n---\n".join(context_blocks)

    #Create message containing query and context
    messages = [
        {
            "role": "system",
            "content": (
                "You are an AI assistant. Use ONLY the provided context to answer. "
                "If the answer is not in the context, say you don't know.\n\n"
                f"CONTEXT:\n{context}"
            )
        },
        {
            "role": "user",
            "content": query
        }
    ]
    #Get answer
    response = client.chat.completions.create(
        model=model_name,   # or your Azure deployment name
        messages=messages,
        max_tokens=400,
        temperature=0.1
    )
    answer = response.choices[0].message.content
    return answer

In [19]:
query = "How to fix my router?"
k=5
search_fields = 'embeddingTitle, embeddingContent'
selected_fields = ['title','content']
docs = search_documents(query, embedding_client, embedding_model_name, search_client, search_fields, selected_fields, k)
answer = generate_answer(docs, client, model_name)
print(answer)


To fix your router, you can follow these general troubleshooting steps:

1. Verify Hardware Connections:
   - Ensure the router is properly connected to the modem using an Ethernet cable (modem to router’s WAN port).
   - Check that both the modem and router are powered on and properly connected to the internet source.

2. Restart Devices:
   - Power cycle the modem by unplugging it, waiting 30 seconds, and plugging it back in.
   - After the modem restarts, do the same for the router.

3. Access Router Settings:
   - Connect your computer or device to the router via Ethernet or Wi-Fi.
   - Open a web browser and enter the router’s IP address (commonly 192.168.0.1 or 192.168.1.1) in the address bar.
   - Log in using the default username and password (often found on a sticker on the router).

4. Check Router Settings:
   - Ensure the router’s WAN settings are set to acquire an IP address automatically (DHCP).
   - If using a static IP, verify the settings are correct.

5. Update Firmwa